In [9]:
pip install openpyxl


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:


print(data.head())
print(data.info())
print(data.describe())


   Year      Y    PC     PB     YD  TEMP    PRM
0  1974  39.70  42.3  143.8  50.10   -16  107.8
1  1975  38.69  49.4  152.2  54.98    -4  134.6
2  1976  42.02  45.5  145.7  59.72   -24  134.0
3  1977  42.71  45.3  145.9  65.17    16  125.4
4  1978  44.75  49.3  178.8  72.24     5  143.6
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    29 non-null     int64  
 1   Y       29 non-null     float64
 2   PC      29 non-null     float64
 3   PB      29 non-null     float64
 4   YD      29 non-null     float64
 5   TEMP    29 non-null     int64  
 6   PRM     29 non-null     float64
dtypes: float64(5), int64(2)
memory usage: 1.7 KB
None
              Year          Y         PC          PB          YD       TEMP  \
count    29.000000  29.000000  29.000000   29.000000   29.000000  29.000000   
mean   1988.000000  65.124828  47.537931  247.917241  153.130345

In [15]:
import os
import itertools
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan, linear_reset
from statsmodels.stats.stattools import durbin_watson
import numpy as np


In [16]:
filename = "chicken_demand_data.xlsx"
if not os.path.exists(filename):
    raise FileNotFoundError(f"File not found at: {filename}. Put the file in this folder or set 'filename' to the correct path.")


In [17]:
df = pd.read_excel(filename)
print("Loaded data. Columns:", df.columns.tolist())
print(df.head())


Loaded data. Columns: ['Year', 'Y', 'PC', 'PB', 'YD', 'TEMP', 'PRM']
   Year      Y    PC     PB     YD  TEMP    PRM
0  1974  39.70  42.3  143.8  50.10   -16  107.8
1  1975  38.69  49.4  152.2  54.98    -4  134.6
2  1976  42.02  45.5  145.7  59.72   -24  134.0
3  1977  42.71  45.3  145.9  65.17    16  125.4
4  1978  44.75  49.3  178.8  72.24     5  143.6


In [18]:
response = "Y"
predictors = ["PC", "PB", "YD", "TEMP", "PRM"]

missing = set([response] + predictors) - set(df.columns)
if missing:
    raise ValueError(f"Missing expected columns in dataset: {missing}")


In [19]:
y = df[response]
X_full = sm.add_constant(df[predictors])


In [20]:
full_model = sm.OLS(y, X_full).fit()
print("\n=== Full model summary ===")
print(full_model.summary())



=== Full model summary ===
                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     576.4
Date:                Wed, 15 Oct 2025   Prob (F-statistic):           2.31e-23
Time:                        22:29:02   Log-Likelihood:                -53.451
No. Observations:                  29   AIC:                             118.9
Df Residuals:                      23   BIC:                             127.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         29.0647   

In [21]:
print("\n=== Coefficients (t-tests & p-values) ===")
print(full_model.summary2().tables[1])



=== Coefficients (t-tests & p-values) ===
           Coef.  Std.Err.          t         P>|t|     [0.025     0.975]
const  29.064718  3.483486   8.343572  2.069165e-08  21.858577  36.270858
PC     -0.109785  0.032593  -3.368401  2.653967e-03  -0.177209  -0.042362
PB      0.038942  0.018232   2.135962  4.355128e-02   0.001227   0.076658
YD      0.247492  0.022033  11.232905  8.163225e-11   0.201914   0.293070
TEMP   -0.017312  0.020937  -0.826857  4.168144e-01  -0.060623   0.026000
PRM    -0.028967  0.030288  -0.956388  3.488214e-01  -0.091622   0.033688


In [22]:
print("\nOverall F-statistic p-value:", full_model.f_pvalue)



Overall F-statistic p-value: 2.3088199708481914e-23


In [23]:
def compute_vif(X_df):
    vif_df = pd.DataFrame({
        "variable": X_df.columns,
        "VIF": [variance_inflation_factor(X_df.values, i) for i in range(X_df.shape[1])]
    })
    return vif_df

vif = compute_vif(X_full)
print("\n=== VIFs (includes constant) ===")
print(vif)



=== VIFs (includes constant) ===
  variable         VIF
0    const  119.477929
1       PC    1.387982
2       PB    9.024920
3       YD   20.928891
4     TEMP    3.662607
5      PRM   18.468309


In [24]:
bp = het_breuschpagan(full_model.resid, full_model.model.exog)
# bp returns: lm_stat, lm_pvalue, f_stat, f_pvalue
print("\nBreusch-Pagan LM p-value:", bp[1], "  f-test p-value:", bp[3])



Breusch-Pagan LM p-value: 0.07106021809312595   f-test p-value: 0.06183578568011185


In [25]:
robust_res = full_model.get_robustcov_results(cov_type='HC3')
print(robust_res.summary())


                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     649.6
Date:                Wed, 15 Oct 2025   Prob (F-statistic):           5.91e-24
Time:                        22:29:43   Log-Likelihood:                -53.451
No. Observations:                  29   AIC:                             118.9
Df Residuals:                      23   BIC:                             127.1
Df Model:                           5                                         
Covariance Type:                  HC3                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         29.0647      3.588      8.100      0.0

In [26]:
dw = durbin_watson(full_model.resid)
print("Durbin-Watson:", dw, " (2 ~ no autocorr; <2 positive autocorr; >2 negative autocorr)")


Durbin-Watson: 0.9409301500503313  (2 ~ no autocorr; <2 positive autocorr; >2 negative autocorr)


In [27]:
try:
    reset = linear_reset(full_model, power=2, use_f=True)
    reset_p = reset.pvalue if hasattr(reset, "pvalue") else (reset[1] if len(reset)>1 else None)
except Exception as e:
    reset_p = np.nan
print("Ramsey RESET p-value:", reset_p)


Ramsey RESET p-value: 0.19122377146643738


In [28]:
results = []
all_predictor_sets = []
for k in range(1, len(predictors)+1):
    for combo in itertools.combinations(predictors, k):
        all_predictor_sets.append(combo)

for combo in all_predictor_sets:
    Xc = sm.add_constant(df[list(combo)])
    model = sm.OLS(y, Xc).fit()
    # RESET
    try:
        r = linear_reset(model, power=2, use_f=True)
        reset_p = r.pvalue if hasattr(r, "pvalue") else (r[1] if len(r)>1 else None)
    except Exception:
        reset_p = np.nan
    # BP p-value
    try:
        bp_c = het_breuschpagan(model.resid, model.model.exog)
        bp_p = bp_c[1]
    except Exception:
        bp_p = np.nan
    results.append({
        "predictors": "+".join(combo),
        "k": len(combo),
        "nobs": int(model.nobs),
        "adj_R2": model.rsquared_adj,
        "AIC": model.aic,
        "BIC": model.bic,
        "F_pvalue": model.f_pvalue,
        "RESET_p": reset_p,
        "BP_p": bp_p,
        "DW": durbin_watson(model.resid)
    })

comp_df = pd.DataFrame(results).sort_values("AIC").reset_index(drop=True)
print("\n=== Top 10 models by AIC ===")
print(comp_df.head(10))
print("\n=== Top 10 models by adj_R2 ===")
print(comp_df.sort_values("adj_R2", ascending=False).head(10))
outpath = "chicken_model_comparison.csv"
comp_df.to_csv(outpath, index=False)
print("\nSaved model comparison to:", outpath)



=== Top 10 models by AIC ===
          predictors  k  nobs    adj_R2         AIC         BIC      F_pvalue  \
0           PC+PB+YD  3    29  0.990455  117.038849  122.508033  5.546770e-26   
1       PC+PB+YD+PRM  4    29  0.990488  117.752380  124.588859  1.112619e-24   
2      PC+PB+YD+TEMP  4    29  0.990396  118.033847  124.870326  1.249960e-24   
3              PC+YD  2    29  0.989600  118.664392  122.766279  6.357399e-27   
4  PC+PB+YD+TEMP+PRM  5    29  0.990361  118.902898  127.106673  2.308820e-23   
5         PC+YD+TEMP  3    29  0.989316  120.306436  125.775619  2.267320e-25   
6          PC+YD+PRM  3    29  0.989266  120.442715  125.911898  2.404443e-25   
7     PC+YD+TEMP+PRM  4    29  0.988931  122.150619  128.987098  6.858128e-24   
8                 YD  1    29  0.987047  124.124731  126.859323  3.081484e-27   
9             YD+PRM  2    29  0.986767  125.649234  129.751122  1.455843e-25   

    RESET_p      BP_p        DW  
0  0.213711  0.117312  1.000757  
1  0.15387